### 1.1 &nbsp; Data Revise & Cleansing
- We combine original data with new additional data crawled and perform data cleansing

### 1.1.1 Data combination & pre-processing

In [1]:
import pandas as pd
import numpy as np
import os

#### Load original datasets and further crawled additional data

In [12]:
df_info_origin = pd.read_csv('./Partial_Datasets/Book_info_original.csv')
df_reviews_origin = pd.read_csv('./Partial_Datasets/Book_reviews_original.csv')

In [3]:
df_info_add = pd.read_csv('./Partial_Datasets/Book_add_info.csv')
df_reviews_add = pd.read_csv('./Partial_Datasets/Book_add_reviews.csv')

#### Also we re-crawled the genre list under each book's information page to replace original ones

In [4]:
uid_genre = pd.read_csv('./Partial_Datasets/uid_genre.csv')
uid_genre

,Uid,Genres
0,13642600,"['Picture Books', 'Biography', 'Art', 'Nonfict..."
1,470185,"['Art', 'Nonfiction', 'Photography', 'Philosop..."
2,137933,"['Art', 'Philosophy', 'Nonfiction', 'Art Histo..."
3,52374,"['Essays', 'Nonfiction', 'Philosophy', 'Art', ..."
4,140987,"['Art', 'Nonfiction', 'Essays', 'Music', 'Crit..."
...,...,...
5659,203818,"['Religion', 'Reference', 'Christianity', 'Cla..."
5660,821535,"['Religion', 'Judaism', 'Nonfiction', 'Classic..."
5661,290882,"['Religion', 'Philosophy', 'Spirituality', 'Cl..."
5662,646462,"['Religion', 'Islam', 'Nonfiction', 'Classics'..."


#### First drop duplicates detected by `Uid` and drop previous genres-related attributes

In [13]:
df_info = df_info_origin.drop_duplicates(subset=['Uid']).reset_index(drop=True)
df_info.drop(['Genre', 'Sub_Genres'], axis = 1, inplace=True)

In [14]:
# combine with new genre information obtained
df_info = df_info.merge(uid_genre, on='Uid')

In [15]:
# adjust column names
cols = df_info.columns.to_list()
cols = cols[:2] + cols[-1:] + cols[2:-1]

In [16]:
df_info = df_info[cols]
df_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5664 entries, 0 to 5663
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Uid            5664 non-null   int64  
 1   Title          5664 non-null   object 
 2   Genres         5664 non-null   object 
 3   Author         5664 non-null   object 
 4   Rating         5664 non-null   float64
 5   Publish_Date   5664 non-null   object 
 6   Page_Num       5664 non-null   int64  
 7   Award          5664 non-null   object 
 8   Description    5653 non-null   object 
 9   Author_Desc    5250 non-null   object 
 10  Book_Authored  5664 non-null   object 
 11  Follower_Num   5655 non-null   object 
 12  Review_Num     5664 non-null   int64  
 13  Rating_Num     5664 non-null   int64  
 14  Rating_Dist    5664 non-null   object 
dtypes: float64(1), int64(4), object(10)
memory usage: 708.0+ KB


#### Combine origianl book information with new additional data

In [17]:
# rename Genre column for combination of dataframes
df_info_add.rename(columns={'Genre': 'Genres'}, inplace=True)

# combine original books with addtional data
df_info = pd.concat([df_info, df_info_add], ignore_index=True)

# again drop duplicates
df_info = df_info.drop_duplicates(subset=['Uid']).reset_index(drop=True)

In [18]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7846 entries, 0 to 7845
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Uid            7846 non-null   int64  
 1   Title          7846 non-null   object 
 2   Genres         7846 non-null   object 
 3   Author         7846 non-null   object 
 4   Rating         7846 non-null   float64
 5   Publish_Date   7846 non-null   object 
 6   Page_Num       7846 non-null   int64  
 7   Award          7846 non-null   object 
 8   Description    7831 non-null   object 
 9   Author_Desc    7185 non-null   object 
 10  Book_Authored  7846 non-null   object 
 11  Follower_Num   7830 non-null   object 
 12  Review_Num     7846 non-null   int64  
 13  Rating_Num     7846 non-null   int64  
 14  Rating_Dist    7846 non-null   object 
dtypes: float64(1), int64(4), object(10)
memory usage: 919.6+ KB


#### Combine original reviews with new additional reviews

In [20]:
df_reviews = pd.concat([df_reviews_origin, df_reviews_add], ignore_index=True)

In [21]:
Uid = df_info['Uid'].values

# only keep books with Uid left in dataset
df_reviews = df_reviews[df_reviews['Uid'].isin(Uid)]
df_reviews = df_reviews.drop_duplicates(subset=['Uid', 'Title', 'Reviewer']).reset_index(drop=True)

In [24]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115033 entries, 0 to 115032
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Uid            115033 non-null  int64 
 1   Title          115033 non-null  object
 2   Reviewer       115033 non-null  object
 3   N_Review       115033 non-null  object
 4   N_Follower     115033 non-null  object
 5   Review_Rating  115033 non-null  int64 
 6   Review_Date    115033 non-null  object
 7   Content        115033 non-null  object
 8   N_Likes        111756 non-null  object
 9   N_Comments     68456 non-null   object
dtypes: int64(2), object(8)
memory usage: 8.8+ MB


In [28]:
# store raw datasets

df_reviews.to_csv('./Partial_Datasets/Book_reviews_combined_raw.csv', encoding='utf-8-sig', index=False)
df_info.to_csv('./Partial_Datasets/Book_info_combined_raw.csv', encoding='utf-8-sig', index=False)

---

### 1.1.2 &nbsp; Clean Book Basic infomation dataframe: `df_info`

### `Genres`

- Note these entries are ***strings***, rather than a list it looks like, so for easier manipulation, we transform them into list of strings

In [55]:
df_info = pd.read_csv('./Partial_Datasets//Book_info_combined_raw.csv')
df_reviews = pd.read_csv('./Partial_Datasets/Book_reviews_combined_raw.csv')

In [30]:
def genre_name_modify(text):
    text_list = re.findall("\'(.*?)\'", text)
    text_cleaned = []
    for text in text_list:
        text = str.lower(text)
        text = text.replace(' ', '-')
        text = text.replace('\'', '-')
        if text == 'childrens':
            text = 'children-s'
        if text == 'n/a':
            text = 'other'
        text_cleaned.append(text)

    return text_cleaned

#### First we drop books whose `Genres` = ['N/A']

In [37]:
uid_list = df_info[df_info['Genres'] == '[\'N/A\']']['Uid'].values
uid_list

array([17383989,   763047, 20701984,    19330,  2153405, 37590384,
          81941,   219024,   233637,   780072, 16144853,   524878],
      dtype=int64)

In [38]:
df_info = df_info[~df_info['Uid'].isin(uid_list)].reset_index(drop=True)

In [39]:
df_reviews = df_reviews[~df_reviews['Uid'].isin(uid_list)].reset_index(drop=True)

In [40]:
import re

genres_df = df_info['Genres'].apply(genre_name_modify)
genres_df

0       [picture-books, biography, art, nonfiction, ch...
1       [art, nonfiction, photography, philosophy, ess...
2       [art, philosophy, nonfiction, art-history, the...
3       [essays, nonfiction, philosophy, art, criticis...
4       [art, nonfiction, essays, music, criticism, me...
                              ...                        
7829    [manga, fantasy, horror, graphic-novels, comic...
7830    [manga, fantasy, graphic-novels, young-adult, ...
7831    [manga, fantasy, graphic-novels, shonen, comic...
7832    [manga, fantasy, graphic-novels, comics, horro...
7833    [manga, fantasy, historical-fiction, supernatu...
Name: Genres, Length: 7834, dtype: object

In [41]:
df_info['Genres'] = genres_df

### `Award`

- The case is a little bit different from before, we can see more messy patterrns, e.g. punctuation (: ' -), different quotations('' ""), varing empty spaces. Therefore, it's hard to write a uniform regular expression to format the strings as wanted

- In this case, we use a more complicated cleansing method as follows:

In [42]:
Award = df_info['Award']

print(Award[0])
print(len(Award[0]))

['Schneider Family Book Award', "Vermont's Picture Book Awards: Red Clover", 'Sibert Medal', "Pennsylvania Young Readers' Choice Award", 'NCTE Orbis Pictus Award']
163


#### Remove punctuation, but keep comma for further separation & turn into lower case to avoid case-sensitive duplicates

In [43]:
from string import punctuation

def remove_punctuation_lower(document):
    keep_comma = punctuation.replace(',', '')
    no_punct = ''.join([character for character in document if character not in keep_comma])
    no_punct_lower = str.lower(no_punct)
    
    return no_punct_lower

In [44]:
Award = Award.apply(remove_punctuation_lower)
Award

0       schneider family book award, vermonts picture ...
1                                                      na
2                                                      na
3                            national book award finalist
4                                                      na
                              ...                        
7829                                                   na
7830                                                   na
7831                                                   na
7832                                                   na
7833                                                   na
Name: Award, Length: 7834, dtype: object

#### Split on comma kept from before & turn combined string into separate award tags in a list

In [45]:
def process_award(tags):
    tags_list = tags.split(',')
    cleaned_tags = []
    
    for item in tags_list:
        item = item.strip()
        item = item.replace(' ', '_')
        cleaned_tags.append(item)
    
    # replace 'NA' by 'no' to represent no award has obtained for easier use
    if cleaned_tags == ['na']:
        cleaned_tags = ['no']
    
    return cleaned_tags 

In [46]:
Award = Award.apply(process_award)
Award

0       [schneider_family_book_award, vermonts_picture...
1                                                    [no]
2                                                    [no]
3                          [national_book_award_finalist]
4                                                    [no]
                              ...                        
7829                                                 [no]
7830                                                 [no]
7831                                                 [no]
7832                                                 [no]
7833                                                 [no]
Name: Award, Length: 7834, dtype: object

In [47]:
df_info['Award'] = Award

###  `Follower_Num`

- Some entries are in the form of thousands (using k), e.g. 13.1k = 13100 in real numbers. We turn them into same format and cast types to int64

In [48]:
df_info['Follower_Num'][18]

'13.1k'

In [49]:
def digit_process(x):
    # some cases, it will be nan
    if pd.isna(x):
        x = 0
    
    if 'k' in str(x):
        x = float(x.split('k')[0])  # ensure if no numeric part, will become xx.0
        x = str(x).replace('.', '')
        x = int(x) * 100
        
    return int(float(x))

In [50]:
df_info['Follower_Num'] = df_info['Follower_Num'].apply(digit_process)
df_info['Follower_Num'][18]

13100

### `Rating_Dist`

In [51]:
df_info['Rating_Dist'][0]

"['1894 (36%)', '1732 (33%)', '866 (16%)', '351 (6%)', '329 (6%)']"

In [52]:
dist = []
for index, row in df_info['Rating_Dist'].iteritems():
    num_list = re.findall("\'(\d*) \(", row)
    dist_temp = []
    for num in num_list:
        num = int(num)
        dist_temp.append(num)
    
    dist.append(dist_temp)

In [53]:
rating_df = pd.DataFrame(data=dist, columns=['Five_Star', 'Four_Star', 'Three_Star', 'Two_Star', 'One_Star'])
rating_df

,Five_Star,Four_Star,Three_Star,Two_Star,One_Star
0,1894,1732,866,351,329
1,3446,2345,1346,395,254
2,368,319,183,41,26
3,3048,3155,1300,257,65
4,1009,745,370,116,73
...,...,...,...,...,...
7829,5932,1694,676,220,178
7830,5556,1603,371,33,9
7831,5342,1429,319,50,13
7832,6996,2997,764,76,25


In [54]:
# drop original rating distribution strings, concatenate cleaned ratings data
df_info.drop(['Rating_Dist'], axis=1, inplace=True)
df_info = pd.concat([df_info, rating_df], axis=1)

### 1.1.3 &nbsp; Book reviews dataframe `df_reviews`

### `N_Review`, `N_Follower`

- Similar as before, nan -> 0; change 'k' into thousand; drop suffixes like 'follower', 'like'

In [55]:
print(df_reviews.iloc[9643, 3])
print(df_reviews.iloc[4300, 4])

995
151


In [56]:
def digit_process2(x):
    # some cases, it will be nan
    if pd.isna(x):
        x = 0
    
    if 'k' in str(x):
        x = float(x.split('k')[0])  # ensure if no numeric part, will become xx.0
        x = str(x).replace('.', '')
        x = int(x) * 10 ** 2
        
    # some data like '1 follower'
    if 'follower' in str(x):
        x = str(x).split(' follower')[0]
    
    if ',' in str(x):
        x = str(x).replace(',', '')

    return int(x)

In [57]:
df_reviews[['N_Review', 'N_Follower']] = df_reviews[['N_Review', 'N_Follower']].applymap(digit_process2)

print(df_reviews.iloc[9643, 3])
print(df_reviews.iloc[4300, 4])

995
151


### `N_Likes`, `N_Comments`

In [58]:
def digit_process3(x):
    # some cases, it will be nan
    if pd.isna(x):
        x = 0
    
    if ',' in str(x):
        x = str(x).replace(',', '')
    
    # modify enties in form: '1 like', '1 comment'
    if 'like' in str(x):
        x = str(x).split(' like')[0]
    
    if 'comment' in str(x):
        x = str(x).split(' comment')[0]
    
    # some tricky entry '3.0'
    if '.0' in str(x):
        x = str(x).replace('.0', '')
        
    return int(x)

In [59]:
df_reviews[['N_Likes', 'N_Comments']] = df_reviews[['N_Likes', 'N_Comments']].applymap(digit_process3)

### 1.1.4 &nbsp; Additional tricky cleansings 

### `Author`

- We will drop some entries where *Author_Name* are *Anonymous* / *Unknown*
- One interesting case is **William Shakespeare***, who authored ***12.4k*** books on GoodReads website, we guess this is due to website wrongly classifying all books that written by other authors but with the same name into his lists. In whatever case, we can think of this as outlier to delete it.

In [60]:
author = ['William Shakespeare', 'Anonymous', 'Unknown', 'Arthur Conan Doyle']

uid_list = df_info[df_info['Author'].isin(author)]['Uid'].values
uid_list

array([ 280111,  930470, 5883777, 1923820,    1622,  102868,   42607,
         52357,    1420,   13006,   12938,    8852,   12957,   12996,
         18135,    3049,    3590,   93101,  188572,   19351,    8921,
        194373,   24128,  608474,   47021,   12985,    1625,  754713,
        194366,  736131,  159964,  534289,   42038,  569564,  118252,
        608947,  790550,  203818,  821535,  290882,  646462,  241902,
        104837, 5031805,   37526,  265205,   42058,  901111,  616330,
        162823,  381112,  463877,  103390,   44133], dtype=int64)

In [61]:
df_info = df_info[~df_info['Author'].isin(author)].reset_index(drop=True)

In [62]:
df_reviews = df_reviews[~df_reviews['Uid'].isin(uid_list)].reset_index(drop=True)

### `Book_Authored`

- As a result of above cleansing on authors, we can cast `Book_Authored` into int64 type now (otherwise will cause error at '12.4k')

In [63]:
df_info['Book_Authored'] = df_info['Book_Authored'].astype('int64')

#### Some books are not in English, so we did not scraped useful reviews for them and we drop these books for simplicity

In [65]:
Uid = df_reviews['Uid'].unique()
len(Uid)

7776

In [66]:
df_info = df_info[df_info['Uid'].isin(Uid)]

In [67]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7776 entries, 0 to 7779
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Uid            7776 non-null   int64  
 1   Title          7776 non-null   object 
 2   Genres         7776 non-null   object 
 3   Author         7776 non-null   object 
 4   Rating         7776 non-null   float64
 5   Publish_Date   7776 non-null   object 
 6   Page_Num       7776 non-null   int64  
 7   Award          7776 non-null   object 
 8   Description    7761 non-null   object 
 9   Author_Desc    7116 non-null   object 
 10  Book_Authored  7776 non-null   int64  
 11  Follower_Num   7776 non-null   int64  
 12  Review_Num     7776 non-null   int64  
 13  Rating_Num     7776 non-null   int64  
 14  Five_Star      7776 non-null   int64  
 15  Four_Star      7776 non-null   int64  
 16  Three_Star     7776 non-null   int64  
 17  Two_Star       7776 non-null   int64  
 18  One_Star

#### Drop duplicated records: we detect duplicates by 3 key factors
- Sometimes a reviewer will review a book several times, so we also use Review_Date as a reference

In [69]:
df_reviews = df_reviews.drop_duplicates(subset=['Uid', 'Reviewer', 'Review_Date'])

In [70]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114034 entries, 0 to 114062
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Uid            114034 non-null  int64 
 1   Title          114034 non-null  object
 2   Reviewer       114034 non-null  object
 3   N_Review       114034 non-null  int64 
 4   N_Follower     114034 non-null  int64 
 5   Review_Rating  114034 non-null  int64 
 6   Review_Date    114034 non-null  object
 7   Content        114034 non-null  object
 8   N_Likes        114034 non-null  int64 
 9   N_Comments     114034 non-null  int64 
dtypes: int64(6), object(4)
memory usage: 9.6+ MB


#### Store the final cleaned datasets

In [71]:
df_info.to_csv('./Book_info_cleaned.csv', index=False)
df_reviews.to_csv('./Book_reviews_cleaned.csv', index=False)